<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/edge-tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------- #
#       INSTALLATION      #
# ----------------------- #

# Upgrade pip to the latest version
!pip install --upgrade pip

# Install Edge-TTS for text-to-speech
!pip install edge-tts

# Install pydub for audio processing
!pip install pydub

# Install ipywidgets for interactive widgets
!pip install ipywidgets

# Install ffmpeg for audio conversion (required by pydub)
!apt-get install ffmpeg -y

# Install nest_asyncio to allow nested event loops
!pip install nest_asyncio

# ----------------------- #
#       IMPORTS           #
# ----------------------- #

import asyncio
import nest_asyncio
import edge_tts
from pydub import AudioSegment
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, Audio
import os

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# ----------------------- #
#   DEFINE AVAILABLE VOICES#
# ----------------------- #

# List of available voices (customize this list based on your preferences)
available_voices = [
    "en-US-GuyNeural",
    "en-US-JennyNeural",
    "en-GB-RyanNeural",
    "en-IE-ConorNeural",
    "es-ES-AlvaroNeural",
    "fr-FR-DeniseNeural",
    "de-DE-KatjaNeural",
    "ja-JP-HarukaNeural",
    "zh-CN-YunxiNeural",
    "ko-KR-SunHiNeural",
    "it-IT-ElsaNeural",
    "pt-BR-RaulNeural",
    # Add more voices as desired
]

# ----------------------- #
#     CREATE WIDGETS      #
# ----------------------- #

# Text input widget
text_input = widgets.Textarea(
    value='Hello! This is a highly natural-sounding speech using Edge-TTS in Google Colab.',
    placeholder='Type your text here...',
    description='Input Text:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='150px')
)

# Voice selection dropdown
voice_dropdown = widgets.Dropdown(
    options=available_voices,
    value=available_voices[0],  # Default to the first voice
    description='Select Voice:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# Generate Speech button
button = widgets.Button(
    description="Generate Speech",
    button_style='success',
    tooltip='Click to generate speech from the input text',
    icon='volume-up'
)

# Output widget for messages and audio
output = widgets.Output()

# Display the widgets
display(text_input, voice_dropdown, button, output)

# ----------------------- #
#    ASYNCHRONOUS TTS      #
# ----------------------- #

async def synthesize_speech(text, voice, output_path):
    """
    Asynchronous function to synthesize speech using Edge-TTS.

    Args:
        text (str): The text to convert to speech.
        voice (str): The selected voice for synthesis.
        output_path (str): The path to save the synthesized audio.
    """
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_path)

# ----------------------- #
#    BUTTON CALLBACK      #
# ----------------------- #

def on_button_clicked(b):
    with output:
        output.clear_output()  # Clear previous outputs
        text = text_input.value.strip()
        selected_voice = voice_dropdown.value

        if not text:
            print("❌ Please enter some text to convert to speech.")
            return

        try:
            print("🔄 Generating speech... Please wait.")
            output_path = "output.mp3"

            # Run the asynchronous synthesis
            asyncio.create_task(synthesize_speech(text, selected_voice, output_path))

            # Post-processing coroutine to handle download and playback
            async def post_process():
                # Wait until the file is created
                while not os.path.exists(output_path):
                    await asyncio.sleep(0.5)
                print("✅ Speech generated successfully!")

                # Provide download link
                files.download(output_path)

                # Display the audio player
                display(Audio(output_path, autoplay=True))

            asyncio.create_task(post_process())

        except Exception as e:
            print(f"❌ An error occurred: {e}")

# Attach the callback to the button
button.on_click(on_button_clicked)